# **Running Jupyter Code with Ollama Server**

1. **Download and Install Ollama**: Ensure that Ollama is downloaded and installed on your system.

2. **Check Ollama Version**:
   ```bash
   ollama --version
   ```

3. **Run Llama 3.2 Model**:
   To load the Llama 3.2 model:
   ```bash
   ollama run llama3.2
   ```

4. **Start Ollama Server**:
   To start the Ollama server, run:
   ```bash
   ollama serve
   ```

**Note**: If you face any issues while running the server, checking the background tasks using the port can help resolve conflicts. Always ensure the port is free before starting the server.

1. **Check if Ollama Server is Running**:
   Check if the server is running and using port 11434:
   ```bash
   netstat -ano | findstr :11434
   ```

2. **Identify and Kill Background Task (if needed)**:
   If you see that another task is using the port, you can identify and kill it:
   ```bash
   tasklist /FI "PID eq <PID>"
   taskkill /PID <PID> /F
   ```

3. **Close Ollama from Windows Task Bar (Optional)**:
   If the server is still not responding or you see issues, you can use the **up arrow key** (^) on the Windows task bar (bottom-left) to close any running Ollama instances.

4. **Start Jupyter Notebook**:
   Once the server is running, start your Jupyter notebook with the necessary code to interact with Ollama.

In [12]:
import ollama
from PyPDF2 import PdfReader

In [16]:
system_prompt = '''You are an agent which analyses the resume and understands the context of resume and 
    extract the following fields and format the output:
    - Name
    - Email
    - Phone 1
    - Phone 2
    - Address
    - City
    - LinkedIn
    - Professional Experience (in years)
    - Highest Education
    - Is Fresher (yes/no)
    - Is Student (yes/no)
    - Skills (comma-separated)
    - Applied For Profile
    - Education (Institute Name, Year of Passing, Score)
    - Projects (Project Name, Description)
    - Professional Experience (Organisation Name, Duration, Profile)

    Ensure to format the output as:
    Name: [Value]
    Email: [Value]
    Phone 1: [Value]
    ... and provide this summary of resume in markdown format'''

In [17]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, "rb") as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text += page.extract_text()  # Extract text from each page
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    return text
    
user_prompt = extract_text_from_pdf('./data/python-developer-resume-example.pdf') + "Provide a summary of this resume"

In [18]:
import requests
import json
from IPython.display import display, Markdown

url = "http://localhost:11434/api/chat"
headers = {
    "Content-Type": "application/json"
}

# Define the conversation prompt and specify the model
data = {
    "model": "llama3.2",  # Specify the model name here
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
}

# Send the POST request to the Ollama server
response = requests.post(url, json=data, headers=headers, stream=True)

# Check if the response is successful
if response.status_code == 200:
    # Initialize a variable to accumulate the assistant's response
    full_response = ""
    
    # Iterate over the chunks of data as they arrive
    for chunk in response.iter_lines():
        if chunk:  # Ignore empty lines
            try:
                # Each chunk is a part of the message, so we can parse it as JSON
                chunk_data = json.loads(chunk.decode('utf-8'))
                
                # Extract the message content
                message_content = chunk_data.get('message', {}).get('content', '')
                full_response += message_content
                
                # Check if the response is done
                if chunk_data.get('done', False):
                    break
                
            except json.JSONDecodeError as e:
                print("Error parsing JSON chunk:", e)
    
    # Display the model's response as Markdown
    display(Markdown(full_response))
else:
    print(f"Error {response.status_code}: {response.text}")


### Resume Summary

#### Contact Information
- **Name:** G I U L I A
- **Email:** ggonzalez@email.com
- **Phone 1:** (123) 456-7890
- **Address:** Detroit, MI
- **City:** Detroit, MI
- **LinkedIn:** LinkedIn
- **Phone 2:** Not provided

#### Professional Experience
- **Professional Experience (Years):** 5 years
- **Highest Education:** M.S. in Computer Science from University of Chicago
- **Education (Institute Name, Year of Passing, Score):**
  - University of Pittsburgh (2010-2014)
  - University of Chicago (2014-2016)

#### Skills
- HTML/CSS
- SQL (PostgreSQL, Oracle)
- JavaScript (Angular)
- Python (Django)
- REST APIs (GraphQL)
- AWS (Redshift, S3)
- Git

#### Work Experience
- **Python Developer**, DoorDash (September 2017 - Present)
    - Built new Angular components for the customer-facing web app, improving time on page by 2 minutes.
    - Collaborated with an agile team to prioritize and scope feature requests, ensuring impact features were worked on first.
    - Built extensive test coverage, reducing customer complaints by 23%.
    - Communicated with internal teams and stakeholders, determining solutions for the user experience.

- **Python Developer Intern**, Knewton (April 2016 - April 2017)
    - Implemented RESTful APIs in Django, increasing reporting speed by 24% for the internal analytics team.
    - Built a unit testing infrastructure using Selenium, reducing bugs reported by 11% month over month.
    - Provided project updates to leadership and offered recommendations for design.

#### Projects
- **Cryptocurrency Price Tracker**
    - Incorporated API calls to several applications, storing data efficiently in PostgreSQL backend.
    - Utilized D3.js to allow users to dynamically visualize price movements over time periods of their choosing.

## **OLD CODE** 

In [19]:
import requests

url = "http://localhost:11434/api/chat"
headers = {
    "Content-Type": "application/json"
}

# Define the conversation prompt and specify the model
data = {
    "model": "llama3.2",  # Specify the model name here
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ]
}

# Send the POST request to the Ollama server
response = requests.post(url, json=data, headers=headers)

# Print the raw response text for debugging
print("Raw Response:", response.text)

# Try to parse the response only if it's in JSON format
try:
    response_json = response.json()  # Try parsing the response as JSON
    print("Model Response:", response_json)
except ValueError as e:
    print("Error parsing JSON:", e)
    print("Response text may not be valid JSON.")


Raw Response: {"model":"llama3.2","created_at":"2024-12-18T09:52:40.7042198Z","message":{"role":"assistant","content":"The"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T09:52:40.8116449Z","message":{"role":"assistant","content":" capital"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T09:52:40.9195623Z","message":{"role":"assistant","content":" of"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T09:52:41.0716295Z","message":{"role":"assistant","content":" France"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T09:52:41.234114Z","message":{"role":"assistant","content":" is"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T09:52:41.3777785Z","message":{"role":"assistant","content":" Paris"},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T09:52:41.4916704Z","message":{"role":"assistant","content":"."},"done":false}
{"model":"llama3.2","created_at":"2024-12-18T09:52:41.6135776Z","message":{"role":"assistant","content

Just Kept the old code to show how Ollama provdes answers it sends one word at a time in a chunk of response 